# Data Pull

## Importing Libraries

In [1]:
import pandas as pd
import os
import requests
from datetime import datetime
from requests import get
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from tqdm import tqdm

## Perform Web Scrape from Craigslist in Los Angeles

**A Note on the Default Search**

The default starting page will have certain characteristics already standardized for the purposes of this project. I have narrowed the focus to only 1 bedroom, 1 bathroom apartments. There is also a specific map area being used. The map view is an area centered around Santa Monica, and encompasses the West Los Angeles region north of Manahattan Beach and south of Pacific Palisades. This area contains a high density of apartments, giving us a steady supply of data to pull.

This strategy pulls many results and isolates two big cost factors in the forthcoming regression equation. This allows us to better view the effects of engineered features, which we will perform later. 

Our goal here is to cater the data science insights to me (the author) first with an eye towards scaling to a potential use case by anyone. Thus, while we will engineer the data infrastructure with an eye towards scaling the data quantity and features, for now we want to narrow the insights to be useful to at least one person (myself) before we expand further. This lets us behave pragmatically within the time constraints of a 7-week project. 

#### First, we declare global variables that will be used in our code.

We use an if statement that will see if the data already exists. It should already exist for purposes of re-running this for a user as the "craigslist_data" was created with an initial batch. The entire set of records takes a long time to pull from Craigslist, so batches after the initial one will only use data that was posted today. This is usually around 100 records. The if statement is used so that we can keep the code in that was used to pull the initial data. Assuming we are using the data from the GitHub, the if statement will return only today's data URL. 

In [2]:
## Global Variables ##

# Declare an 'if' statement to decide which records should be pulled
if os.path.exists('../Data/refreshed_craigslist_data.csv'):
    print('Core data already exists. Pulling today\'s records only.')
    craigslist_base_url = 'https://losangeles.craigslist.org/search/santa-monica-ca/apa?lat=34.0315&lon=-118.461&max_bathrooms=1&max_bedrooms=1&min_bathrooms=1&min_bedrooms=1&postedToday=1&search_distance=3.6#search=1'
    craigslist_search_first_page_url = 'https://losangeles.craigslist.org/search/santa-monica-ca/apa?lat=34.0315&lon=-118.461&max_bathrooms=1&max_bedrooms=1&min_bathrooms=1&min_bedrooms=1&postedToday=1&search_distance=3.6#search=1~list~0~0'
    
else:
    print('Data does not exist. Pulling all records.')
    craigslist_base_url = 'https://losangeles.craigslist.org/search/santa-monica-ca/apa?lat=34.0315&lon=-118.461&max_bathrooms=1&max_bedrooms=1&min_bathrooms=1&min_bedrooms=1&postal=90095&search_distance=3.6#search=1'
    craigslist_search_first_page_url = 'https://losangeles.craigslist.org/search/santa-monica-ca/apa?lat=34.0315&lon=-118.461&max_bathrooms=1&max_bedrooms=1&min_bathrooms=1&min_bedrooms=1&postal=90095&search_distance=3.6#search=1~list~0~0'
    
chrome_driver_path = '../Other_Material/chromedriver-mac-arm64/chromedriver'

Core data already exists. Pulling today's records only.


#### We will be using BeautifulSoup to access URLs in this notebook, so we write a function to perform this operation now.

In [3]:
# Create the access_beautiful_soup function
def access_beautiful_soup(url):
    # Call a get instance with the URL
    response = requests.get(url)

    # Sleep in order to not overwhelm servers
    time.sleep(5 + 10 * random.random())

    # Find all the listings links on the page
    soup = BeautifulSoup(response.text, 'html.parser')

    return soup

#### Before we access the URLs of the listings, we need to find out how many results are in the search query. 

For the search in the area of Los Angeles we are doing, the number is typically around 2400 at any given time (posts expire after 30 days). However, we want to construct the application with an eye towards scaling, so we need to make the infrastructure flexible for different result amounts.

Since this information is not accessible via BeautifulSoup based on the way the Craigslist HTML structure is set up, we need to use Selenium. 

We now proceed with implementing the Selenium code to get the total listings amount. We write a function called "get_postings_count" that takes in the two arguments "website" and "path" and returns the post count. We need to use JavaScript here, because the post count is loaded dynamically into the webpage. 

In [4]:
# Function to return the number of posts at a given time
def get_postings_count(website, path):
    
    # # prevent a window from opening in Selenium
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    
    # set up the Chrome driver path for Selenium usage
    service = Service(path)
    driver = webdriver.Chrome(service=service, options=options)

    # Call a "get" instance of the initial Craigslist page to initialize Selenium
    driver.get(website)

    # Put in a function stopper to let the page render.
    # 15 seconds should be plenty, but if the result is coming back as
    # "no results," the first troubleshoot would be to increase this time
    # to see if that fixes it.
    time.sleep(15)  

    try:

        # Use JavaScript to set up a script to return the postings count
        postings_count_script = """
            var postingsDiv = document.querySelector('.cl-count-save-bar > div');
            return postingsDiv ? postingsDiv.textContent : 'Postings count not found';
        """

        # Execute the script to get the post count and return it
        postings_count = driver.execute_script(postings_count_script)

        # Quit Selenium
        driver.quit()

        # Return the postings count
        return postings_count
    except Exception as e:
        print(f"Error encountered: {e}")

#### Let's call the function now to get the postings count.

In [5]:
# Call the get_postings_count function
postings_count = get_postings_count(craigslist_search_first_page_url, chrome_driver_path)

#### Finally, we print the amount of posts to see how many we are working with.

In [6]:
# Check to see how many postings there are
print(postings_count)

64 postings


#### We can now use the post count to get the number of pages to loop through when we extract the listings. 

There are 120 posts per page, so we want to extract the post count and divide it by 120.

In [7]:
# Function to calculate the number of pages for us to loop through
def calculate_pages_from_postings(postings_count_str):
    
    # Remove commas and extract the numerical part of the string
    num_postings = int(postings_count_str.replace(" postings", "").replace(",", ""))
    
    # 120 posts per page
    postings_per_page = 120
    
    # Calculate the number of pages needed to display all postings, accounting for remainder
    num_pages = -(-num_postings // postings_per_page)  
    
    return num_pages

In [8]:
# Call the calculate_pages_from_postings function
number_of_pages = calculate_pages_from_postings(postings_count)
print(f"Number of pages: {number_of_pages}")

Number of pages: 1


#### Next, we write a function to extract the links from each of the pages. We do this by: 

1. initializing a list
2. looping through the number of pages
3. finding all the 'li' tags within page that use the class 'cl-static-search-result'
4. finding the 'a' tags within the 'li' tags, which contain the link to the individual listing
5. appending these links to the list

In [9]:
def extract_listing_links(path, base_url, number_of_pages):
    all_listing_links = []
    
    for page_number in range(number_of_pages):
        page_url = f'{base_url}~list~{page_number}~0'
        
        # prevent a window from opening in Selenium
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        
        # set up the Chrome driver path for Selenium usage
        service = Service(path)
        driver = webdriver.Chrome(service=service, options=options)
        
        driver.get(page_url)
        
        # Wait for the listings to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.cl-search-result.cl-search-view-mode-list"))
        )
        # Now that the page is loaded, find all the `a` tags within the listings
        listing_links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, "li.cl-search-result.cl-search-view-mode-list a")]

        all_listing_links.extend(listing_links)

        driver.quit()
        
    return all_listing_links

In [10]:
all_links = extract_listing_links(chrome_driver_path, craigslist_base_url, number_of_pages)

#### Let's check out the "all_links" list to see the extraction was successful.

In [11]:
print(len(all_links))

64


#### We see that there is content within the all_links list. 

Let us also get a sample of three of the links to ensure we pulled what we wanted.

In [12]:
print(f'Link #1: ',all_links[0])
print(f'Link #2: ',all_links[1])
print(f'Link #3: ',all_links[2])

Link #1:  https://losangeles.craigslist.org/wst/apa/d/los-angeles-remodeled-bedroom-suite/7737006536.html
Link #2:  https://losangeles.craigslist.org/wst/apa/d/los-angeles-vinyl-plank-flooring/7737017231.html
Link #3:  https://losangeles.craigslist.org/wst/apa/d/los-angeles-specials-furnished-co/7737015440.html


#### We now dive into working with the data within each listing link.

First, we set up a dataframe containing basic column information.

In [13]:
# Initialize the DataFrame
df_columns = ["Title", "Price", "Bedrooms", "Square Feet", "Full Address"]
todays_listings_df = pd.DataFrame(columns=df_columns)

# Set the max columns to infinite so that we may view all of them
pd.set_option('display.max_columns', None)

## We declare an object called "links_and_soups" to pair each link with its BeautifulSoup content.

This is a lengthy process, due to the fact that we have random sleep intervals between each time we access BeautifulSoup. 

We need to do this because the soup content will contain the information we want, so we need the soup content for each of the links. 

We start by declaring the object that will hold the links and soups as key value pairs.

## NOTE TO SELF DO NOT RUN BELOW CELL

In [14]:
# # Load the CSV file
# df = pd.read_csv("../Data/linksnsoups.csv", header=None, names=['link', 'soup_string'])

In [15]:
links_and_soups = {}

In [16]:
# # Convert string back to BeautifulSoup objects and populate the dictionary
# for index, row in df.iterrows():
#     soup_object = BeautifulSoup(row['soup_string'], 'html.parser')  # Assuming 'html.parser' was used initially
#     links_and_soups[row['link']] = soup_object

# print(f"Loaded {len(links_and_soups)} entries.")

## NOTE TO SELF DO NOT RUN ABOVE CELL

#### We write a function for pairing the links with their soup content.

In [17]:
# Function to pair links with soup content
def pair_links_and_soups(list_of_links):
    for link in list_of_links:
        the_soup = access_beautiful_soup(link)
        links_and_soups[link] = the_soup

## NOTE TO SELF DO NOT RUN BELOW CELL

We run the function using the links that were pulled previously.

**NOTE: This can take a long time. For example, 2300 links took 7.5 hours**

In [18]:
pair_links_and_soups(all_links)

## NOTE TO SELF DO NOT RUN ABOVE CELL

#### Let's check to see if the links and soups object was successfully populated with data.

In [19]:
print(len(links_and_soups))

64


#### Next, we begin the process of creating boolean values for different attributes.

Each listing contains different attributes that the poster uses to convey information about a property and market it. While there is a lot of overlap between listings, we need to see all of the options. To do this, we initialize a dictionary called "full_attribute_counts," then add unique values and count them. Ultimately, we want to create columns with these values and use boolean values "1" or "0" meaning "present" or "not present" in the listing.

In [20]:
global_attribute_counts = {}

In [21]:
# Define the count_attributes_function to view all the attributes used in apartment listings
def process_attributes(the_soup):
    attribute_search = the_soup.find_all('div', class_='attr')
    attributes = []
    fee_needed = 0  # Initialize a flag for fees

    fee_pattern = re.compile(r'\b\d+\b')  # Regex to identify fee-related attributes

    for listing in attribute_search:
        value_span = listing.find('span', class_='valu')
        if value_span:
            attribute = value_span.text.strip()
            global_attribute_counts[attribute] = global_attribute_counts.get(attribute, 0) + 1 
            if fee_pattern.search(attribute):  # Check if attribute suggests a fee
                fee_needed = 1
            else:
                attributes.append(attribute)  # Only add non-fee attributes to the list

    return attributes, fee_needed

#### Below we run the process_attributes function

In [22]:
# Run process_attributes using the info in the links_and_soups dictionary
for link, soup in links_and_soups.items():
    attributes = process_attributes(soup)

#### Let's take a look at all the attributes that were pulled.

In [23]:
# Storing the object results in a variable called "raw_attributes" 
raw_attributes = global_attribute_counts

In [24]:
raw_attributes

{'monthly': 64,
 'cats are OK - purrr': 50,
 'apartment': 64,
 'dogs are OK - wooof': 46,
 'laundry in bldg': 23,
 'detached garage': 12,
 'air conditioning': 50,
 'EV charging': 22,
 'laundry on site': 18,
 'off-street parking': 20,
 'w/d in unit': 23,
 '$55- Non Refundable': 1,
 'carport': 14,
 'no smoking': 18,
 'wheelchair accessible': 20,
 'attached garage': 16,
 'valet parking': 1,
 'furnished': 7,
 'no parking': 1,
 '$52': 1,
 '$49.50': 2}

#### We see a lot of overlap but also noise to clean up
We see there is a high proportion of overlap with "monthly" being the most common item found. Outside the most common, there is some noise to clean up. Looking closer, we see that the noise is mostly made up of application and credit check fees. To clean this up, we can remove all of this and simply group all of these into a key called "Fee Needed To Apply". Let's write a function that will clean up listings with fee information. This will do a very simple check: if there is an attribute with an integer in it, then it will be categorized as a fee. 

In [25]:
# Function to group together fee related attributes
def clean_up_the_fees(attributes_dictionary):
    
    # Initialize a count for "Fees Needed To Apply Key"
    fees_needed_to_apply = 0

    # Set up a Regex to identify keys containing integers
    # We will use the re package to do this
    fee_pattern = re.compile(r'\b\d+\b')

    # Iterate through the dictionary, summing up counts for fee-related attributes
    for key, value in raw_attributes.items():
        if fee_pattern.search(key):
            fees_needed_to_apply += value
    
    # Update the dictionary and add in a key called "Fee Needed To Apply"
    cleaned_attributes = {key: value for key, value in raw_attributes.items() if not fee_pattern.search(key)}
    cleaned_attributes["Fee Needed To Apply"] = fees_needed_to_apply

    return cleaned_attributes

In [26]:
# Run the clean_up_the_fees function using the raw_attributes as input
cleaned_attributes = clean_up_the_fees(raw_attributes)

In [27]:
# Sort the cleaned_attributes in descending order of instance count
cleaned_attributes = dict(sorted(cleaned_attributes.items(), key=lambda item: item[1], reverse=True))

#### Inspecting the cleaned attributes dictionary.

We see the fee related material is now grouped into one key called "Fee Needed To Apply," such that we answer the question of whether or not a fee is needed for an application.

In [28]:
cleaned_attributes

{'monthly': 64,
 'apartment': 64,
 'cats are OK - purrr': 50,
 'air conditioning': 50,
 'dogs are OK - wooof': 46,
 'laundry in bldg': 23,
 'w/d in unit': 23,
 'EV charging': 22,
 'off-street parking': 20,
 'wheelchair accessible': 20,
 'laundry on site': 18,
 'no smoking': 18,
 'attached garage': 16,
 'carport': 14,
 'detached garage': 12,
 'furnished': 7,
 'Fee Needed To Apply': 4,
 'valet parking': 1,
 'no parking': 1}

#### We will have some noise that we don't know the meaning of. 

Many of the attributes seem to be one-off items that are unique to one or two posts. Let us get rid of the ones with less than 5 instances, with "w/d hookups" as our cut-off.

In [29]:
# Filtering out attributes with less than 10 instance counts
filtered_attributes = {key: value for key, value in cleaned_attributes.items() if value >= 5}

In [30]:
filtered_attributes

{'monthly': 64,
 'apartment': 64,
 'cats are OK - purrr': 50,
 'air conditioning': 50,
 'dogs are OK - wooof': 46,
 'laundry in bldg': 23,
 'w/d in unit': 23,
 'EV charging': 22,
 'off-street parking': 20,
 'wheelchair accessible': 20,
 'laundry on site': 18,
 'no smoking': 18,
 'attached garage': 16,
 'carport': 14,
 'detached garage': 12,
 'furnished': 7}

#### Finally, we collect the basic information from the listings and create the dataframe.

In [31]:
def collect_basic_information(the_soup):
    title_element = the_soup.find("span", id="titletextonly")
    title = title_element.text.strip() if title_element else "Title Not Found"
    
    price_element = the_soup.find("span", class_="price")
    price = price_element.text.strip() if price_element else "Price Not Found"
    
    housing_element = the_soup.find("span", class_="housing")
    if housing_element:
        try:
            bedroom_info = housing_element.text.split("/")[1].split("-")[0].strip()
            square_feet = housing_element.text.split("-")[1].split("ft")[0].strip()
        except IndexError:
            bedroom_info = "Bedrooms Info Not Found"
            square_feet = "Square Feet Not Found"
    else:
        bedroom_info = "Bedrooms Info Not Found"
        square_feet = "Square Feet Not Found"
    
    full_address_element = the_soup.find("h2", class_="street-address")
    full_address = full_address_element.text.strip() if full_address_element else "None listed"

    return title, price, bedroom_info, square_feet, full_address


In [32]:
def create_dataframe(links_and_soups, todays_listings_df):
    localized_df = todays_listings_df.copy()
    
    for link, soup in links_and_soups.items():
        title, price, bedroom_info, square_feet, full_address = collect_basic_information(soup)
        listing_attributes, fee_needed = process_attributes(soup)  # Capture fee_needed flag here
        
        # Start with basic info
        new_row_data = {
            "Title": title,
            "Price": price,
            "Bedrooms": bedroom_info,
            "Square Feet": square_feet,
            "Full Address": full_address,
        }
        
        # For each attribute in filtered_attributes, add to new_row_data with 1 or 0
        for attribute in filtered_attributes.keys():
            new_row_data[attribute] = 1 if attribute in listing_attributes else 0
        
        # Add "Fee Needed To Apply" after processing filtered_attributes
        new_row_data["Fee Needed To Apply"] = fee_needed

        # Convert new_row_data to a DataFrame row and concat to localized_df
        new_row_df = pd.DataFrame([new_row_data])
        localized_df = pd.concat([localized_df, new_row_df], ignore_index=True)
    
    return localized_df


### Combining Dataframes

#### We now need to combine the existing data with the newest data. 

Below is a series of steps to achieve this. Part of the process is filling in the gaps caused by a smaller sample (one day's worth) of data. We then need to match the column ordering between dataframes. Then, we input the date that the new data was added and then combine the two dataframes.

In [33]:
# Now call the updated function
todays_listings_df = create_dataframe(links_and_soups, todays_listings_df)

In [34]:
todays_listings_df.head()

,Title,Price,Bedrooms,Square Feet,Full Address,monthly,apartment,cats are OK - purrr,air conditioning,dogs are OK - wooof,laundry in bldg,w/d in unit,EV charging,off-street parking,wheelchair accessible,laundry on site,no smoking,attached garage,carport,detached garage,furnished,Fee Needed To Apply
0,Remodeled 1 Bedroom Suite + Loft in Culver Ci...,"$3,045",1br,800,"2600 Overland Avenue, West Los Angeles, CA 90064",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Vinyl Plank Flooring + Refrigerator + Lovely 1...,"$1,900",1br,550,"3258 Overland Avenue, Palms, CA 90034",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SPECIALS ^ Furnished Co-living Near UCLA ^ Sub...,"$2,744",1br,160,"1533 Selby Avenue, Los Angeles, CA 90024",1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,SPECIALS - 1 Bedroom 1 Bath in Westwood - Cent...,"$2,995",1br,674,"520 Kelton Avenue, Los Angeles, CA 90024",1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Newly Remodeled 1 Bedroom 1 BA in Mar Vista + ...,"$2,395",1br,780,"3749 McLaughlin Ave, West Los Angeles, CA 90066",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
len(todays_listings_df)

64

In [38]:
existing_df = pd.read_csv('../Data/refreshed_craigslist_data.csv')

In [39]:
len(existing_df)

2593

In [40]:
# Display the columns of the existing DataFrame
existing_df_columns = existing_df.columns
existing_df_columns

Index(['Title', 'Price', 'Bedrooms', 'Square Feet', 'Full Address', 'monthly',
       'apartment', 'cats are OK - purrr', 'dogs are OK - wooof',
       'laundry on site', 'air conditioning', 'off-street parking',
       'EV charging', 'w/d in unit', 'carport', 'no smoking',
       'attached garage', 'detached garage', 'laundry in bldg',
       'Fee Needed To Apply', 'wheelchair accessible', 'no parking',
       'furnished', 'street parking', 'no laundry on site', 'house',
       'w/d hookups', 'date_added'],
      dtype='object')

In [41]:
# Identify missing columns
missing_columns = [col for col in existing_df_columns if col not in todays_listings_df.columns]

In [42]:
missing_columns

['no parking',
 'street parking',
 'no laundry on site',
 'house',
 'w/d hookups',
 'date_added']

In [43]:
# Identify missing columns
missing_columns2 = [col for col in todays_listings_df.columns if col not in existing_df_columns]

In [44]:
missing_columns2

[]

In [45]:
# Add missing columns with default value
for col in missing_columns:
    todays_listings_df[col] = 0 

In [46]:
todays_listings_df.head()

,Title,Price,Bedrooms,Square Feet,Full Address,monthly,apartment,cats are OK - purrr,air conditioning,dogs are OK - wooof,laundry in bldg,w/d in unit,EV charging,off-street parking,wheelchair accessible,laundry on site,no smoking,attached garage,carport,detached garage,furnished,Fee Needed To Apply,no parking,street parking,no laundry on site,house,w/d hookups,date_added
0,Remodeled 1 Bedroom Suite + Loft in Culver Ci...,"$3,045",1br,800,"2600 Overland Avenue, West Los Angeles, CA 90064",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,0,0
1,Vinyl Plank Flooring + Refrigerator + Lovely 1...,"$1,900",1br,550,"3258 Overland Avenue, Palms, CA 90034",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,SPECIALS ^ Furnished Co-living Near UCLA ^ Sub...,"$2,744",1br,160,"1533 Selby Avenue, Los Angeles, CA 90024",1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,0,0,0
3,SPECIALS - 1 Bedroom 1 Bath in Westwood - Cent...,"$2,995",1br,674,"520 Kelton Avenue, Los Angeles, CA 90024",1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4,Newly Remodeled 1 Bedroom 1 BA in Mar Vista + ...,"$2,395",1br,780,"3749 McLaughlin Ave, West Los Angeles, CA 90066",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0


In [47]:
existing_df.head()

,Title,Price,Bedrooms,Square Feet,Full Address,monthly,apartment,cats are OK - purrr,dogs are OK - wooof,laundry on site,air conditioning,off-street parking,EV charging,w/d in unit,carport,no smoking,attached garage,detached garage,laundry in bldg,Fee Needed To Apply,wheelchair accessible,no parking,furnished,street parking,no laundry on site,house,w/d hookups,date_added
0,Title Not Found,Price Not Found,Bedrooms Info Not Found,Square Feet Not Found,None listed,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/20/24
1,1 Bedroom in the Heart of Venice* Plank Floors...,"$2,895",1br,750,"237 Fourth Avenue, Venice, CA 90291",1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3/20/24
2,"SPECIALS, Rooftop Sky Deck, Brand New 1+1 Bren...","$3,438",1br,711,"11916 West Pico Boulevard, Los Angeles, CA 90064",1,1,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,3/20/24
3,1 Bedroom 1 Bath Westwood Apartment in Westwoo...,"$3,175",1br,NaN,"972 Hilgard Ave, Los Angeles, CA 90024",1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,3/20/24
4,"Dishwasher, Efficient Appliances, 1 Bed","$2,895",1br,575,"1720 Pacific Ave, Venice, CA 90291",1,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3/20/24


In [48]:
column_order = existing_df.columns.tolist()

In [49]:
column_order

['Title',
 'Price',
 'Bedrooms',
 'Square Feet',
 'Full Address',
 'monthly',
 'apartment',
 'cats are OK - purrr',
 'dogs are OK - wooof',
 'laundry on site',
 'air conditioning',
 'off-street parking',
 'EV charging',
 'w/d in unit',
 'carport',
 'no smoking',
 'attached garage',
 'detached garage',
 'laundry in bldg',
 'Fee Needed To Apply',
 'wheelchair accessible',
 'no parking',
 'furnished',
 'street parking',
 'no laundry on site',
 'house',
 'w/d hookups',
 'date_added']

In [50]:
todays_listings_df = todays_listings_df[column_order]

In [51]:
todays_listings_df.head()

,Title,Price,Bedrooms,Square Feet,Full Address,monthly,apartment,cats are OK - purrr,dogs are OK - wooof,laundry on site,air conditioning,off-street parking,EV charging,w/d in unit,carport,no smoking,attached garage,detached garage,laundry in bldg,Fee Needed To Apply,wheelchair accessible,no parking,furnished,street parking,no laundry on site,house,w/d hookups,date_added
0,Remodeled 1 Bedroom Suite + Loft in Culver Ci...,"$3,045",1br,800,"2600 Overland Avenue, West Los Angeles, CA 90064",1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0.0,0,0,0,0,0
1,Vinyl Plank Flooring + Refrigerator + Lovely 1...,"$1,900",1br,550,"3258 Overland Avenue, Palms, CA 90034",1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0
2,SPECIALS ^ Furnished Co-living Near UCLA ^ Sub...,"$2,744",1br,160,"1533 Selby Avenue, Los Angeles, CA 90024",1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0
3,SPECIALS - 1 Bedroom 1 Bath in Westwood - Cent...,"$2,995",1br,674,"520 Kelton Avenue, Los Angeles, CA 90024",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0
4,Newly Remodeled 1 Bedroom 1 BA in Mar Vista + ...,"$2,395",1br,780,"3749 McLaughlin Ave, West Los Angeles, CA 90066",1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0


In [52]:
existing_df.head()

,Title,Price,Bedrooms,Square Feet,Full Address,monthly,apartment,cats are OK - purrr,dogs are OK - wooof,laundry on site,air conditioning,off-street parking,EV charging,w/d in unit,carport,no smoking,attached garage,detached garage,laundry in bldg,Fee Needed To Apply,wheelchair accessible,no parking,furnished,street parking,no laundry on site,house,w/d hookups,date_added
0,Title Not Found,Price Not Found,Bedrooms Info Not Found,Square Feet Not Found,None listed,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/20/24
1,1 Bedroom in the Heart of Venice* Plank Floors...,"$2,895",1br,750,"237 Fourth Avenue, Venice, CA 90291",1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,3/20/24
2,"SPECIALS, Rooftop Sky Deck, Brand New 1+1 Bren...","$3,438",1br,711,"11916 West Pico Boulevard, Los Angeles, CA 90064",1,1,1,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,3/20/24
3,1 Bedroom 1 Bath Westwood Apartment in Westwoo...,"$3,175",1br,NaN,"972 Hilgard Ave, Los Angeles, CA 90024",1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,3/20/24
4,"Dishwasher, Efficient Appliances, 1 Bed","$2,895",1br,575,"1720 Pacific Ave, Venice, CA 90291",1,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3/20/24


#### It will be useful to see the time data was added.

Thus, we create a function to add this in as a final column.

In [53]:
def add_the_datetime(dataFrame):
    # Generate the current date and time in the specified format
    current_date = datetime.now().strftime('%Y-%m-%d')
    
    # Add a new column to the DataFrame with the current date and time
    dataFrame['date_added'] = current_date

In [54]:
add_the_datetime(todays_listings_df)

In [55]:
todays_listings_df.head()

,Title,Price,Bedrooms,Square Feet,Full Address,monthly,apartment,cats are OK - purrr,dogs are OK - wooof,laundry on site,air conditioning,off-street parking,EV charging,w/d in unit,carport,no smoking,attached garage,detached garage,laundry in bldg,Fee Needed To Apply,wheelchair accessible,no parking,furnished,street parking,no laundry on site,house,w/d hookups,date_added
0,Remodeled 1 Bedroom Suite + Loft in Culver Ci...,"$3,045",1br,800,"2600 Overland Avenue, West Los Angeles, CA 90064",1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0.0,0,0,0,0,2024-04-14
1,Vinyl Plank Flooring + Refrigerator + Lovely 1...,"$1,900",1br,550,"3258 Overland Avenue, Palms, CA 90034",1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,2024-04-14
2,SPECIALS ^ Furnished Co-living Near UCLA ^ Sub...,"$2,744",1br,160,"1533 Selby Avenue, Los Angeles, CA 90024",1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0,0,0,0,2024-04-14
3,SPECIALS - 1 Bedroom 1 Bath in Westwood - Cent...,"$2,995",1br,674,"520 Kelton Avenue, Los Angeles, CA 90024",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,2024-04-14
4,Newly Remodeled 1 Bedroom 1 BA in Mar Vista + ...,"$2,395",1br,780,"3749 McLaughlin Ave, West Los Angeles, CA 90066",1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,2024-04-14


In [56]:
# Step 1: Check if column orders match
if list(existing_df.columns) == list(todays_listings_df.columns):
    
    # Step 2: Combine the DataFrames
    combined_df = pd.concat([existing_df, todays_listings_df], ignore_index=True)
    
    # Step 3: Sort the combined DataFrame by 'date_added' column
    combined_df = combined_df.sort_values(by='date_added', ascending=True)
    
    print("DataFrames combined and sorted successfully.")
else:
    print("Error: Column orders do not match.")


DataFrames combined and sorted successfully.


In [58]:
combined_df.to_csv('../Data/refreshed_craigslist_data.csv', index=False)